In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

In [11]:
data = pd.read_csv('../Q4/data.csv')

In [12]:
data.head()

,episode,robot_pos,target_pos,step
0,0,"[189.27300925719734, 176.0042213800998, 184.0]","[196, 184]",191
1,1,"[588.8211336450494, 211.07850307606793, 142.0]","[599, 212]",632
2,2,"[420.45935323626566, 187.87996701234405, 340.0]","[411, 184]",701
3,3,"[428.0140693681143, 449.6938401830937, 274.0]","[420, 456]",724
4,4,"[359.11187611868274, 61.26133092751468, 72.0]","[364, 52]",1033


In [13]:
data['robot_pos'] = data['robot_pos'].apply(lambda x: x[1:-1].split(','))
data['target_pos'] = data['target_pos'].apply(lambda x: x[1:-1].split(','))

X_robot = np.vstack(data['robot_pos'].apply(lambda x: [float(i) for i in x]))
X_target = np.vstack(data['target_pos'].apply(lambda x: [float(i) for i in x]))

In [14]:
X = np.hstack((X_robot, X_target))
y = data['step'].to_numpy()

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)


In [16]:
## Creating a neural network

import torch
import torch.nn as nn
import torch.optim as optim

class Regression(nn.Module):
    def __init__(self, input_size, output_size):
        super(Regression, self).__init__()
        self.linear1 = nn.Linear(input_size, 128)
        self.linear2 = nn.Linear(128, 64)
        self.linear3 = nn.Linear(64, output_size)
    
    def forward(self, x):
        x = torch.relu(self.linear1(x))
        x = torch.relu(self.linear2(x))
        x = self.linear3(x)
        return x

In [22]:
input_size = X_train.shape[1]

model = Regression(input_size, 1)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)


## Training the model

epochs = 10000
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(torch.tensor(X_train, dtype=torch.float32))
    loss = criterion(outputs, torch.tensor(y_train, dtype=torch.float32).unsqueeze(1))
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}')

## Testing the model

model.eval()  # Set the model to evaluation mode

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

outputs = model(X_test_tensor)

test_loss = criterion(outputs, y_test_tensor)

print(f'Test Loss: {test_loss.item():.4f}')


Epoch 1/10000, Loss: 653762.4375
Epoch 2/10000, Loss: 652441.8750
Epoch 3/10000, Loss: 650800.7500
Epoch 4/10000, Loss: 648663.0000
Epoch 5/10000, Loss: 645978.3750
Epoch 6/10000, Loss: 642674.1875
Epoch 7/10000, Loss: 638676.7500
Epoch 8/10000, Loss: 633906.1250
Epoch 9/10000, Loss: 628278.5000
Epoch 10/10000, Loss: 621707.3750
Epoch 11/10000, Loss: 614102.5625
Epoch 12/10000, Loss: 605371.3750
Epoch 13/10000, Loss: 595427.6250
Epoch 14/10000, Loss: 584182.3750
Epoch 15/10000, Loss: 571550.3125
Epoch 16/10000, Loss: 557454.4375
Epoch 17/10000, Loss: 541826.5625
Epoch 18/10000, Loss: 524612.5625
Epoch 19/10000, Loss: 505776.9375
Epoch 20/10000, Loss: 485306.5938
Epoch 21/10000, Loss: 463217.8438
Epoch 22/10000, Loss: 439562.9688
Epoch 23/10000, Loss: 414439.2500
Epoch 24/10000, Loss: 387995.4375
Epoch 25/10000, Loss: 360441.5312
Epoch 26/10000, Loss: 332058.6562
Epoch 27/10000, Loss: 303209.1562
Epoch 28/10000, Loss: 274345.3750
Epoch 29/10000, Loss: 246019.2344
Epoch 30/10000, Loss: 2

In [23]:
# Calculate the Mean Absolute Error (MAE)
mae = torch.nn.functional.l1_loss(outputs, y_test_tensor).item()
print(f'MAE: {mae:.4f}')

# Calculate the Root Mean Squared Error (RMSE)
mse = torch.nn.functional.mse_loss(outputs, y_test_tensor).item()
rmse = np.sqrt(mse)
print(f'RMSE: {rmse:.4f}')

MAE: 227.8552
RMSE: 286.1057
